## Setup Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

## Create Distiller class with modified compile, train_step, and test_step methods

In [2]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_function = student_loss_fn
        self.distillation_loss_function = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        x, y = data
        
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            student_predictions = self.student(x, training=True)

            student_loss = self.student_loss_function(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_function(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        self.compiled_metrics.update_state(y, student_predictions)

        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        x, y = data

        y_prediction = self.student(x, training=False)

        student_loss = self.student_loss_function(y, y_prediction)

        self.compiled_metrics.update_state(y, y_prediction)

        results = {met.name: met.result() for met in self.metrics}
        results.update({"student_loss": student_loss})
        return results

## Teacher-Student Model Creation

In [3]:
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)

student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

student_scratch = keras.models.clone_model(student)

## Prepare and Preprocess Train and Test Set

In [4]:
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

## Train the Teacher Model

In [5]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 217s 115ms/step - loss: 0.1455 - sparse_categorical_accuracy: 0.9562
Epoch 2/5
1875/1875 [==============================] - 218s 116ms/step - loss: 0.0931 - sparse_categorical_accuracy: 0.9722
Epoch 3/5
1875/1875 [==============================] - 218s 116ms/step - loss: 0.0808 - sparse_categorical_accuracy: 0.9758
Epoch 4/5
1875/1875 [==============================] - 215s 114ms/step - loss: 0.0732 - sparse_categorical_accuracy: 0.9794
Epoch 5/5
313/313 [==============================] - 6s 19ms/step - loss: 0.0841 - sparse_categorical_accuracy: 0.9797


[0.08408331871032715, 0.9797000288963318]

## Initialize and Compile Distiller

In [6]:
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=12,
)

distiller.fit(x_train, y_train, epochs=5)

distiller.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 41s 21ms/step - sparse_categorical_accuracy: 0.9160 - student_loss: 0.3715 - distillation_loss: 8.2367
Epoch 2/5
1875/1875 [==============================] - 40s 21ms/step - sparse_categorical_accuracy: 0.9732 - student_loss: 0.1057 - distillation_loss: 1.8635
Epoch 3/5
1875/1875 [==============================] - 42s 22ms/step - sparse_categorical_accuracy: 0.9782 - student_loss: 0.0855 - distillation_loss: 1.2598
Epoch 4/5
1875/1875 [==============================] - 41s 22ms/step - sparse_categorical_accuracy: 0.9795 - student_loss: 0.0778 - distillation_loss: 0.9842
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - sparse_categorical_accuracy: 0.9774 - student_loss: 0.0908


[0.977400004863739, 4.766300116898492e-05]

## Train and Evaluate Student Model

In [7]:
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

student_scratch.fit(x_train, y_train, epochs=5)
student_scratch.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2398 - sparse_categorical_accuracy: 0.9291
Epoch 2/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0905 - sparse_categorical_accuracy: 0.9719
Epoch 3/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0748 - sparse_categorical_accuracy: 0.9770
Epoch 4/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0649 - sparse_categorical_accuracy: 0.9800
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0544 - sparse_categorical_accuracy: 0.9822


[0.054402440786361694, 0.982200026512146]